In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [517 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,144 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,165 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,016 k

In [9]:
# Import packages
import pandas as pd
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [12]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

df=pd.read_csv(url)

In [13]:

# 2. Create a temporary view of the DataFrame.
# Convert the pandas DataFrame to a Spark DataFrame
spark_df = spark.createDataFrame(df)

# Create a temporary view of the DataFrame
spark_df.createOrReplaceTempView("home_sales")

# Show the first 20 rows of the Spark DataFrame
spark_df.show(20)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [14]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
results = spark.sql(
    "SELECT year(date) as year, ROUND(AVG(price), 2) as avg_price FROM home_sales WHERE bedrooms = 4 GROUP BY year(date) ORDER BY YEAR(date)")
results.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [15]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
results2 = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY YEAR(date)
    ORDER BY YEAR(date)
""")

results2.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
|2022|292725.69|
+----+---------+



In [16]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

results3 = spark.sql("""
SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE Bedrooms = 3 AND Bathrooms = 3 AND Floors = 2 AND SQFT_living >= 2000
GROUP BY YEAR(date)
ORDER BY YEAR(date)
""")

results3.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019|289859.14|
|2020|292289.09|
|2021|296330.96|
|2022|290242.99|
+----+---------+



In [21]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

result4 = spark.sql("""
SELECT ROUND(AVG(price), 2) AS avg_price, AVG(view) AS avg_view
FROM home_sales
WHERE price >= 350000
""")

result4.show()

print("--- %s seconds ---" % (time.time() - start_time))

+---------+--------+
|avg_price|avg_view|
+---------+--------+
|473796.26|  32.264|
+---------+--------+

--- 0.6759729385375977 seconds ---


In [22]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache().createOrReplaceTempView("home_cache")

In [23]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [25]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

result5 = spark.sql("""
SELECT ROUND(AVG(price), 2) AS avg_price, AVG(view) AS avg_view
FROM home_cache
WHERE price >= 350000
""")

result5.show()

print("--- %s seconds ---" % (time.time() - start_time))


+---------+--------+
|avg_price|avg_view|
+---------+--------+
|473796.26|  32.264|
+---------+--------+

--- 0.42440104484558105 seconds ---


In [26]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
import pandas as pd

# Load the CSV file into a Pandas DataFrame
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
df = pd.read_csv(url)

# Convert the pandas DataFrame to a Spark DataFrame
spark_df = spark.createDataFrame(df)

# Create a temporary view of the DataFrame
spark_df.createOrReplaceTempView("home_sales")

# Cache the temporary table home_sales
spark.catalog.cacheTable("home_sales")

# Convert the temporary table to a dataframe
home_cache = spark.table("home_sales")

# Write the cached dataframe as partitioned parquet
home_cache.write.partitionBy("date_built").mode("overwrite").parquet("home_partitioned")

In [27]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("home_partitioned")

In [28]:
# 12. Create a temporary table for the parquet data.
# Read the parquet formatted data
parquet_data = spark.read.parquet("home_partitioned")

# Create a temporary view of the parquet data
parquet_data.createOrReplaceTempView("parquet_sales")

In [30]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

result6 = spark.sql("""
    SELECT ROUND(AVG(price), 2) AS avg_price, AVG(view) AS avg_view
FROM parquet_sales
WHERE price >= 350000
""")

result6.show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------+--------+
|avg_price|avg_view|
+---------+--------+
|473796.26|  32.264|
+---------+--------+

--- 0.5583500862121582 seconds ---


In [31]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [32]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False